# This notebook requires a deployment of "gpt-4o-mini", edit as needed

## Web Search Ant As Tool

This example demonstrates one ant using another ant as a tool so that the second ant's work is outside the main thread.

The **WebSearchAgentAnt** demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

WebSearchAgentAnt uses **WebPageAgentAnt** via a tool call to evaluate search results using the search results link.
This demo uses a lot of tokens. 

**If your quota is low, you will get throttled**

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}
Environment.SetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini");

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.5

In [2]:
var assistantId = await AssistantUtility.Create("WebSearchAgentAnt", config);
Console.WriteLine(assistantId);
assistantId = await AssistantUtility.Create("WebPageAgentAnt", config);
Console.WriteLine(assistantId);

asst_ptT9wLT5a5UodStYZK0SW54j
asst_PqMKdTYgD1uO6S8ki7rn0rOY


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().date()
today_date
```

I called the tool named crawl with these arguments:
```
{"q":"things to do in Atlanta on 2024-12-31","count":5,"textFormat":"Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-12-31"},"webPages":{"value":[{"name":"Atlanta Events in December 2024 - Schedule \u0026 Tickets","url":"https://atlanta-ga.events/december/","displayUrl":"https://atlanta-ga.events/december","snippet":"Check Out The Atlanta Events Calendar for December 2024: Current \u0026 Upcoming Events \u002B Best \u0026 Fun Thing To Do in the City. ... Dec 29, 2024 12:30 PM ... Dec 31, 2024 3:30 AM . Georgia Tech Yellow Jackets Basketball . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM . Notre Dame Fighting Irish Basketball ..."},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! DiscoverAtlanta.com homepage. Things to Do ... New Year\u2019s Eve Celebration at Red Phone Booth 2024 12/31/24. New Year\u2019s Eve Dinner at Signia by Hilton Atlanta 12/31/24 \u2013 1/1/25. New Years Eve Party 2025 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"11 Atlanta holiday events happening this weekend | 11alive.com","url":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26","displayUrl":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26...","snippet":"Date and Location: Dec. 25\u201331; 10 a.m.\u20137:30 p.m.; Atlanta Beltline East Side Trail, 700 Lake Avenue NE, Atlanta, GA 30307 Description: Craft a custom soy candle while enjoying snacks, drinks ...","thumbnailUrl":"https://www.bing.com/th?id=OIF.DuWLyAoWt9tU6lqzr3sKvA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - December 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","snippet":"Things to do in Atlanta, GA in December, including upcoming festivals, Christmas events and other holiday celebrations, lights shows, holiday markets, comedy, musicals, ballet, tours of homes, and more. ... \u2013 Anime Weekend Atlanta, December 12-15 2024 ... \u2013 New Year\u2019s Eve Celebration, December 31 2024 New Year\u2019s eve event at the Georgia ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... 12 tickets remaining! Tickets; Dec. 31. 2024. 9:30 PM Tue. Daily Bread. The Eastern - GA. ... Dec 31, 2024 3:30 AM . Notre Dame Fighting Irish . Events: 3. McCamish Pavilion ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on December 31, 2024? https://atlanta-ga.events/december/"}
```
and got this result:
```
"On December 31, 2024, the following events are happening in Atlanta:\n\n1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**\n   - **Time:** 2:30 PM\n   - **Location:** McCamish Pavilion\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/)\n\n2. **Provenance NYE: Seven Lions, Knife Party \u0026 Grabbitz**\n   - **Time:** 7:00 PM\n   - **Location:** Atlanta Coliseum\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/)\n\n3. **Zac Brown Band**\n   - **Time:** 8:00 PM\n   - **Location:** State Farm Arena\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/)\n\n4. **Moon Taxi**\n   - **Time:** 8:00 PM\n   - **Location:** The Tabernacle\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/)\n\n5. **Juelz - Artist**\n   - **Time:** 9:00 PM\n   - **Location:** Believe Music Hall\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/juelz-artist-atlanta-tuesday-december-31-2024-900-pm/)\n\n6. **Gimme Gimme Disco**\n   - **Time:** 9:00 PM\n   - **Location:** Hell at The Masquerade\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/)\n\n7. **The Players Ball NYE Experience: Thrice Groove**\n   - **Time:** 9:00 PM\n   - **Location:** Buckhead Theatre\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/the-players-ball-nye-experience-thrice-groove-atlanta-tuesday-december-31-2024-900-pm/)\n\n8. **Big Freedia**\n   - **Time:** 9:30 PM\n   - **Location:** City Winery\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/)\n\n9. **Daily Bread**\n   - **Time:** 9:30 PM\n   - **Location:** The Eastern\n   - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/)\n\n10. **Hugel - Artist**\n    - **Time:** 10:00 PM\n    - **Location:** District Atlanta\n    - **Tickets:** [Get Tickets](https://atlanta-ga.events/event/hugel-artist-atlanta-tuesday-december-31-2024-1000-pm/)\n\nThese events provide a variety of options for celebrating New Year\u0027s Eve in Atlanta!"
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on December 31, 2024? https://discoveratlanta.com/events/main/"}
```
and got this result:
```
"On December 31, 2024, several events are happening in Atlanta. Here are some highlights:\n\n1. **Deck the Hall at the College Football Hall of Fame**\n   - **Dates**: 12/31/24 \u2013 1/1/25\n   - [Event Link](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)\n\n2. **Noon Year\u2019s Eve Family Celebration**\n   - **Date**: 12/31/24\n   - [Event Link](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)\n\n3. **Midnight at Chateau**\n   - **Dates**: 12/31/24 \u2013 1/1/25\n   - [Event Link](https://discoveratlanta.com/event/detail/midnight-at-chateau/)\n\n4. **NYE @ May Peel \u2013 A Champagne Story**\n   - **Date**: 12/31/24\n   - [Event Link](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)\n\n5. **New Year\u2019s Eve Bash at The Whitley**\n   - **Date**: 12/31/24\n   - [Event Link](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)\n\n6. **New Year\u2019s Eve Black-Tie Dinner at The Waldorf**\n   - **Date**: 12/31/24\n   - [Event Link](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)\n\n7. **New Year\u2019s Eve Black-Tie Gala at The Waldorf**\n   - **Dates**: 12/31/24 \u2013 1/1/25\n   - [Event Link](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-gala-at-the-waldorf/)\n\n8. **Noon Year\u2019s Eve at LEGO Discovery Center Atlanta**\n   - **Date**: 12/31/24\n   - [Event Link](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)\n\n9. **Toast to 2025: Amore e Amore Rings in the New Year**\n   - **Date**: 12/31/24\n   - [Event Link](https://discoveratlanta.com/event/detail/toast-to-2025-amore-e-amore-rings-in-the-new-year/)\n\n10. **Atlanta\u0027s New Year\u2019s Eve Peach Drop Festival**\n    - **Dates**: 12/31/24 \u2013 1/1/25\n    - [Event Link](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)\n\nThese events offer a variety of celebrations, from family-friendly activities to elegant dinners and parties."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on December 31, 2024? https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26"}
```
and got this result:
```
"Here are some events happening in Atlanta on December 31, 2024:\n\n1. **Create Your Own Scented Candle**\n   - **Date and Location:** December 25 \u2013 31, 10 a.m. \u2013 7:30 p.m.; Atlanta Beltline East Side Trail, 700 Lake Avenue NE, Atlanta, GA 30307\n   - **Description:** Craft a custom soy candle while enjoying snacks, drinks, and a relaxing atmosphere.\n   - **Event Details:** [More Info](https://www.atlantacandlemaking.com)\n\n2. **Holiday Bricktacular at LEGO Discovery Center**\n   - **Date and Location:** December 25 \u2013 31; LEGO Discovery Center Atlanta, 3500 Peachtree Road NE, Atlanta, GA 30326\n   - **Description:** Build, play, and celebrate at LEGO Discovery Center\u2019s festive event featuring unique activities and attractions.\n   - **Event Details:** [More Info](https://www.legolanddiscoverycenter.com)\n\nThese events are part of a broader holiday celebration in Atlanta, showcasing a variety of activities for visitors and residents."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on December 31, 2024? https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html"}
```
and got this result:
```
"On December 31, 2024, the following events are happening in Atlanta:\n\n1. **Zoo Year\u2019s Eve**  \n   - A daytime New Year\u2019s Eve event at Zoo Atlanta.  \n   - [More details here](https://zooatlanta.org/event/zoo-years-eve/)\n\n2. **New Year\u2019s Eve Celebration**  \n   - An event at the Georgia Aquarium featuring a DJ and dancing, food, and after-hours access to the aquatic exhibits.  \n   - [More details here](https://www.georgiaaquarium.org/events/event/new-years-eve-2024/)\n\n3. **Resolution Run**  \n   - A New Year\u2019s Day 5K run, along with family races in Piedmont Park.  \n   - [More details here](https://www.atlantatrackclub.org/2025-resolution-run)\n\nThese events offer a mix of family-friendly activities and festive celebrations to ring in the New Year!"
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on December 31, 2024? https://atlanta-ga.events/"}
```
and got this result:
```
"On December 31, 2024, several events are happening in Atlanta:\n\n1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**\n   - **Time**: 2:30 PM\n   - **Location**: McCamish Pavilion\n   - **Tickets**: Prices from $18, Average price ~ $62\n   - [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/?pid=1282)\n\n2. **Provenance NYE: Seven Lions, Knife Party \u0026 Grabbitz**\n   - **Time**: 7:00 PM\n   - **Location**: Atlanta Coliseum\n   - **Tickets**: Prices from $114, Average price ~ $132\n   - [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)\n\n3. **Moon Taxi**\n   - **Time**: 8:00 PM\n   - **Location**: The Tabernacle\n   - **Tickets**: Prices from $65, Average price ~ $127\n   - [Get Tickets](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/?pid=7640)\n\n4. **Zac Brown Band**\n   - **Time**: 8:00 PM\n   - **Location**: State Farm Arena\n   - **Tickets**: Prices from $73, Average price ~ $237\n   - [Get Tickets](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/?pid=769)\n\n5. **Gimme Gimme Disco**\n   - **Time**: 9:00 PM\n   - **Location**: Hell at The Masquerade\n   - **Tickets**: Prices from $49, Average price ~ $63\n   - [Get Tickets](https://atlanta-ga.events/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/?pid=1469)\n\n6. **Juelz - Artist**\n   - **Time**: 9:00 PM\n   - **Location**: Believe Music Hall\n   - **Tickets**: Prices from $69, Average price ~ $79\n   - [Get Tickets](https://atlanta-ga.events/event/juelz-artist-atlanta-tuesday-december-31-2024-900-pm/?pid=4969)\n\n7. **The Players Ball NYE Experience: Thrice Groove**\n   - **Time**: 9:00 PM\n   - **Location**: Buckhead Theatre\n   - **Tickets**: Prices from $44, Average price ~ $51\n   - [Get Tickets](https://atlanta-ga.events/event/the-players-ball-nye-experience-thrice-groove-atlanta-tuesday-december-31-2024-900-pm/?pid=7935)\n\n8. **Big Freedia**\n   - **Time**: 9:30 PM\n   - **Location**: City Winery\n   - **Tickets**: Prices from $224, Average price ~ $258\n   - [Get Tickets](https://atlanta-ga.events/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/?pid=2964)\n\n9. **Daily Bread**\n   - **Time**: 9:30 PM\n   - **Location**: The Eastern\n   - **Tickets**: Prices from $111, Average price ~ $158\n   - [Get Tickets](https://atlanta-ga.events/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/?pid=4699)\n\n10. **Hugel - Artist**\n    - **Time**: 10:00 PM\n    - **Location**: District Atlanta\n    - **Tickets**: Prices from $122, Average price ~ $189\n    - [Get Tickets](https://atlanta-ga.events/event/hugel-artist-atlanta-tuesday-december-31-2024-1000-pm/?pid=2550)\n\nThese events offer a variety of entertainment options, from sports to concerts and special New Year\u0027s Eve parties."
```

Here are some exciting events happening in Atlanta on December 31, 2024, perfect for celebrating New Year's Eve:

### Major Events
1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**
   - **Time:** 2:30 PM
   - **Location:** McCamish Pavilion
   - [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/)

2. **Provenance NYE: Seven Lions, Knife Party & Grabbitz**
   - **Time:** 7:00 PM
   - **Location:** Atlanta Coliseum
   - [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/)

3. **Zac Brown Band**
   - **Time:** 8:00 PM
   - **Location:** State Farm Arena
   - [Get Tickets](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/)

4. **Moon Taxi**
   - **Time:** 8:00 PM
   - **Location:** The Tabernacle
   - [Get Tickets](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/)

5. **Gimme Gimme Disco**
   - **Time:** 9:00 PM
   - **Location:** Hell at The Masquerade
   - [Get Tickets](https://atlanta-ga.events/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/)

### Family-Friendly and Special Events
1. **Zoo Year’s Eve**
   - A daytime New Year’s Eve event at Zoo Atlanta.
   - [More details here](https://zooatlanta.org/event/zoo-years-eve/)

2. **Noon Year’s Eve Family Celebration**
   - A fun event for families to celebrate the New Year early.
   - [Event Link](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)

3. **Atlanta's New Year’s Eve Peach Drop Festival**
   - A festive celebration with entertainment and the iconic peach drop.
   - [Event Link](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)

4. **Deck the Hall at the College Football Hall of Fame**
   - A festive holiday event running through New Year’s.
   - [Event Link](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)

These events provide a variety of options for both adults and families to enjoy the festivities of New Year's Eve in Atlanta. Make sure to check the links for ticket availability and more details!

### Sources
- [Atlanta Events Calendar](https://atlanta-ga.events/december/)
- [Discover Atlanta Events](https://discoveratlanta.com/events/main/)
- [11 Alive Events](https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26)
- [Go South Atlanta Events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html)


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events happening in Atlanta on December 31, 2024, perfect for celebrating New Year's Eve:

### Major Events
1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish**
   - **Time:** 2:30 PM
   - **Location:** McCamish Pavilion
   - [Get Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/)

2. **Provenance NYE: Seven Lions, Knife Party & Grabbitz**
   - **Time:** 7:00 PM
   - **Location:** Atlanta Coliseum
   - [Get Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/)

3. **Zac Brown Band**
   - **Time:** 8:00 PM
   - **Location:** State Farm Arena
   - [Get Tickets](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/)

4. **Moon Taxi**
   - **Time:** 8:00 PM
   - **Location:** The Tabernacle
   - [Get Tickets](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/)

5. **Gimme Gimme Disco**
   - **Time:** 9:00 PM
   - **Location:** Hell at The Masquerade
   - [Get Tickets](https://atlanta-ga.events/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/)

### Family-Friendly and Special Events
1. **Zoo Year’s Eve**
   - A daytime New Year’s Eve event at Zoo Atlanta.
   - [More details here](https://zooatlanta.org/event/zoo-years-eve/)

2. **Noon Year’s Eve Family Celebration**
   - A fun event for families to celebrate the New Year early.
   - [Event Link](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)

3. **Atlanta's New Year’s Eve Peach Drop Festival**
   - A festive celebration with entertainment and the iconic peach drop.
   - [Event Link](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)

4. **Deck the Hall at the College Football Hall of Fame**
   - A festive holiday event running through New Year’s.
   - [Event Link](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)

These events provide a variety of options for both adults and families to enjoy the festivities of New Year's Eve in Atlanta. Make sure to check the links for ticket availability and more details!

### Sources
- [Atlanta Events Calendar](https://atlanta-ga.events/december/)
- [Discover Atlanta Events](https://discoveratlanta.com/events/main/)
- [11 Alive Events](https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26)
- [Go South Atlanta Events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html)

In [6]:
assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and correct links from ExtractContentFromUrl.\n\nWhen choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.\nDon't forget to include valid links!",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and correct links from ExtractContentFromUrl.

When choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.
Don't forget to include valid links!

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the next four weekends (Saturday)
weekends = [(today + timedelta(days=(5 - today.weekday() + 7 * i))).date() for i in range(4)]
weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 4 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 4 2025"},"webPages":{"value":[{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. Check out the complete schedule of this month\u2019s events. ... Jan 4, 2025 12:00 PM . Expressions of Shukran . Events: 6. Gas South Theater . Jan 4, 2025 12:00 PM . Kennesaw State Owls Basketball ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 1 To The Weekend Of January 4-5 2025. \u2192 See December events. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns.","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... The Millennium Tour 2025 4/4/25. Georgia Tasting Festival 2025 4/6/25. Spring Break Magic Show ... January 18. Saturday. AT\u0026T Playoff Playlist Live with Lil Wayne \u0026 More! 1/18/25 \u2013 1/19/25.","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events | Explore Georgia","url":"https://exploregeorgia.org/atlanta/events","displayUrl":"https://exploregeorgia.org/atlanta/events","snippet":"Browse the ideas and Atlanta events below to help plan your trip. Find Events. Arts \u0026 Culture (21) Culinary, Food, Wine \u0026 Beer (12) Entertainment (67) Fairs \u0026 Festivals (21) ... January 4, 2025 - May 24, 2025 Painting Zoo Painters. Zoo Atlanta. View all dates. Atlanta. Sunday, January 5. January 5, 2025 - April 27, 2025","thumbnailUrl":"https://www.bing.com/th?id=OIP.qcVhz7_S17NbT6s7cPa8nQHaEi\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Cool Things To Do in February in Atlanta; Cool Things To Do in December in Atlanta; Cool Things to Do in March in Atlanta; Family-Friendly Things To Do. Grab the whole family to explore the city, all while trying to keep warm. Delta Flight Museum. 2025 marks the 100th anniversary of Delta Air Lines.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://www.experism.com/atlanta/january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.NLuvOTPMpK4lHBiF1ixSaQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.kZGqRzEnhMrmwo63tm9cRgHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/cb/b4/0d/cbb40d70b98766feb2db482f8b31a3ff.jpg","hostPageUrl":"https://in.pinterest.com/pin/654710864573177138/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.SEMK2FArnC1F1qLMYZxj6QHaEK\u0026pid=Api","contentUrl":"https://foodtoursatlanta.com/wp-content/uploads/2021/09/Blue-Engagement-Essentials-Blog-Banner.png","hostPageUrl":"https://foodtoursatlanta.com/33-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.MdoK5WF-Bvvg72JvIonreQHaEG\u0026pid=Api","contentUrl":"https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png","hostPageUrl":"https://stonehurstplace.com/our-diary/attractions/things-to-do-in-atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cn277ZSG6rKOabzwcv6aOQHaEo\u0026pid=Api","contentUrl":"https://help.randmcnally.com/images/blog/_750/Skyview_1280x800.jpg","hostPageUrl":"https://wwp.randmcnally.com/blog/post/19-fun-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.0lJD1SVeqMnc5EvrdSDk1wHaNK\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/34/89/3e/34893e1cea1cdaf1dcd8644d411a4018.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/460985711869965247/101-things-to-do-in-atlanta-ga-the-ultimate-atlanta-bucket-list/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.SeKsuJEAl7i4y99RBVdFjAHaE7\u0026pid=Api","contentUrl":"https://tds.indianeagle.com/wp-content/uploads/2023/11/Things-To-Do-in-Atlanta-1.png","hostPageUrl":"https://www.thecommongroundnetwork.com/blog/international-travel/discover-best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.wIKEhD1FU_pWgHXkSv90OwHaLH\u0026pid=Api","contentUrl":"https://familydestinationsguide.com/wp-content/uploads/2022/11/Fun-Things-to-Do-in-Atlanta-at-Night-pinterest.jpg","hostPageUrl":"https://familydestinationsguide.com/fun-things-to-do-in-atlanta-at-night/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.HEoI2TVQ4IQANGVQ6WgRfAHaO0\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/c5/b0/37/c5b037658ec64ee6f0295bce3cc6fb59.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/649996158733928577/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.4SmjXMfeQzC5ARWsf4T0hAHaLG\u0026pid=Api","contentUrl":"https://www.onedayinacity.com/wp-content/uploads/2020/03/Ultimate-Guide-Atlanta-One-Day-683x1024.png","hostPageUrl":"https://www.onedayinacity.com/one-day-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.unmT01rPD7si8w-n3kzOXgHaHa\u0026pid=Api","contentUrl":"https://cdn2.storyasset.link/5f018ed1-7c8d-49e0-9d76-582219d18631/Untitled%20design%20-1-ms-rtheoozqlb.jpg","hostPageUrl":"https://tripononline.com/web-stories/ultimate-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.pTFzMm6LiPCR7Yr9RrSvhwHaE8\u0026pid=Api","contentUrl":"https://tourscanner.com/blog/wp-content/uploads/2020/11/fun-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://tourscanner.com/blog/fun-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6srgUoyHkiOt8iaSj9z8tgAAAA\u0026pid=Api","contentUrl":"https://heyeastcoastusa.com/wp-content/uploads/2022/12/Ferris-Wheel-at-Skyview-Atlanta-in-Georgia-at-Night.Depositphotos_164614584_L.jpg","hostPageUrl":"https://heyeastcoastusa.com/romantic-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.PzRE6SJDtJ9H3Z9jEfK5cAHaE8\u0026pid=Api","contentUrl":"https://image.jimcdn.com/app/cms/image/transf/dimension=640x10000:format=jpg/path/s9524a0afdc5b40b0/image/i88ad01ce4abaee84/version/1483187737/8-fun-things-to-do-in-atlanta-travel-city-guide-via-just1wayticket.jpg","hostPageUrl":"https://www.justonewayticket.com/2016/06/14/8-fun-things-to-do-in-atlanta/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 11 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 11 2025"},"webPages":{"value":[{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. Go South! Atlanta; ... \u2013 Last Podcast on the Left: JK Ultra Tour, January 11 2025 Comedy show at the Coca-Cola Roxy. More details \u2013 Atlanta Travel \u0026 Adventure Show, January 11-12 2025"},{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025 Check out the complete schedule of this month\u2019s events. \u201C Atlanta Events \u201D is your guide to epic events: concerts , theater shows , Broadway musicals and plays, opera , ballet , and sporting events to experience while in town.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"January 11 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/january-11/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/january-11","snippet":"Save this event: 2025 Atlanta Travel \u0026 Adventure Show Share this event: 2025 Atlanta Travel \u0026 Adventure Show. 2025 Atlanta Travel \u0026 Adventure Show. ... Sat, Jan 11, 4:00 PM. Atlanta\u0027s Original Uptown Comedy Corner (225) Save this event: Saturday Shenanigans Open Mic Comedy Show, ..."},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026 Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"Things to do in Atlanta January Atlanta Nightclub Events in January ... Don\u2019t miss the Atlanta\u0027s 2025 Chick-fil-A Peach Bowl on January 1. As one of college football\u2019s premier postseason games, the Peach Bowl brings together two of the nation\u2019s top teams for a gridiron showdown at Mercedes-Benz Stadium. ... Jan 11, 2025 Atlanta View on ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover 2025 Events \u0026 Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Save this event: 2025 Atlanta Travel \u0026 Adventure Show Share this event: 2025 Atlanta Travel \u0026 Adventure Show. 2025 Atlanta Travel \u0026 Adventure Show. ... ELEVATE : IGNITE CONNECTIONS FOR 2025. Sat, Jan 11, 5:00 PM. The Ritz-Carlton, Atlanta. Share this event: ELEVATE : IGNITE CONNECTIONS FOR 2025 Save this event: ELEVATE : IGNITE CONNECTIONS FOR ..."}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.NLuvOTPMpK4lHBiF1ixSaQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.SEMK2FArnC1F1qLMYZxj6QHaEK\u0026pid=Api","contentUrl":"https://foodtoursatlanta.com/wp-content/uploads/2021/09/Blue-Engagement-Essentials-Blog-Banner.png","hostPageUrl":"https://foodtoursatlanta.com/33-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.wsLrsO7mRA1lPtu-yRPZHQAAAA\u0026pid=Api","contentUrl":"https://secretatlanta.co/wp-content/uploads/2021/12/259158507_6597733293632503_4617562253295709670_n.jpg","hostPageUrl":"https://secretatlanta.co/things-to-do-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://www.experism.com/atlanta/january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.SeKsuJEAl7i4y99RBVdFjAHaE7\u0026pid=Api","contentUrl":"https://tds.indianeagle.com/wp-content/uploads/2023/11/Things-To-Do-in-Atlanta-1.png","hostPageUrl":"https://www.thecommongroundnetwork.com/blog/international-travel/discover-best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.DKJGVacYCM6vSxdQBwzLvAHaE8\u0026pid=Api","contentUrl":"https://cdn.thecrazytourist.com/wp-content/uploads/2018/11/ccimage-shutterstock_152538065.jpg","hostPageUrl":"https://www.thecrazytourist.com/25-best-things-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.pTFzMm6LiPCR7Yr9RrSvhwHaE8\u0026pid=Api","contentUrl":"https://tourscanner.com/blog/wp-content/uploads/2020/11/fun-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://tourscanner.com/blog/fun-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6srgUoyHkiOt8iaSj9z8tgAAAA\u0026pid=Api","contentUrl":"https://heyeastcoastusa.com/wp-content/uploads/2022/12/Ferris-Wheel-at-Skyview-Atlanta-in-Georgia-at-Night.Depositphotos_164614584_L.jpg","hostPageUrl":"https://heyeastcoastusa.com/romantic-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.PzRE6SJDtJ9H3Z9jEfK5cAHaE8\u0026pid=Api","contentUrl":"https://image.jimcdn.com/app/cms/image/transf/dimension=640x10000:format=jpg/path/s9524a0afdc5b40b0/image/i88ad01ce4abaee84/version/1483187737/8-fun-things-to-do-in-atlanta-travel-city-guide-via-just1wayticket.jpg","hostPageUrl":"https://www.justonewayticket.com/2016/06/14/8-fun-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.wIKEhD1FU_pWgHXkSv90OwHaLH\u0026pid=Api","contentUrl":"https://familydestinationsguide.com/wp-content/uploads/2022/11/Fun-Things-to-Do-in-Atlanta-at-Night-pinterest.jpg","hostPageUrl":"https://familydestinationsguide.com/fun-things-to-do-in-atlanta-at-night/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.3I8aZp7EItFo_du1fS5CVQHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/dc/81/14/dc8114905086f0025ffc20dfd179934b.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/649996158733928577/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.0lJD1SVeqMnc5EvrdSDk1wHaNK\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/34/89/3e/34893e1cea1cdaf1dcd8644d411a4018.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/460985711869965247/101-things-to-do-in-atlanta-ga-the-ultimate-atlanta-bucket-list/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://martellewgennie.pages.dev/ycqmcw-things-to-do-in-atlanta-this-weekend-events-dgpyjjsmr-photos/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VOzVfWJV7_6joNkGhLP7AAHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/ce/c5/25/cec525ba57be0022de5d70d1a1f3dd88.jpg","hostPageUrl":"https://www.pinterest.com/pin/277393658288987671/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Yac324FtG-dH_KY8XaUTZQHaME\u0026pid=Api","contentUrl":"https://www.frostandsun.com/wp-content/uploads/2017/11/things-to-do-in-atlanta-pin-me-1-628x1024.jpg.webp","hostPageUrl":"https://www.frostandsun.com/things-to-do-in-atlanta/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 18 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 18 2025"},"webPages":{"value":[{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. Check out the complete schedule of this month\u2019s events. ... Sports Events: 18 . McCamish Pavilion . 30332, 965 Fowler St., Atlanta, GA, US . Sports Events: 17 . Mercedes-Benz Stadium . 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. Go South! ... January 18-20 2025. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. Open Menu. ... Atlanta: 18 Jan 2025: Electric Avenue at Variety Playhouse: Variety Playhouse, Atlanta: 18 Jan 2025: Fulton Lee at The Loft at Center Stage:"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Find cool things to do in Atlanta during the month of January, from annual festivals to seasonal activities, theatrical performances, and more! ... The events are happening Jan. 18-20 from 10 a.m. to 6 p.m. in the Georgia World Congress Center Building A. This is free to the public. ... 2025 marks the 100th anniversary of Delta Air Lines. Stop ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Multiple dates until 01/18/2025. Atlanta\u2019s Youth Basketball League 12/29/24 \u2013 12/30/24. Festive Features ... January 18. Saturday. AT\u0026T Playoff Playlist Live with Lil Wayne \u0026 More! 1/18/25 \u2013 1/19/25.","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://www.experism.com/atlanta/january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.wsLrsO7mRA1lPtu-yRPZHQAAAA\u0026pid=Api","contentUrl":"https://secretatlanta.co/wp-content/uploads/2021/12/259158507_6597733293632503_4617562253295709670_n.jpg","hostPageUrl":"https://secretatlanta.co/things-to-do-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.3I8aZp7EItFo_du1fS5CVQHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/dc/81/14/dc8114905086f0025ffc20dfd179934b.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/649996158733928577/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.-1BVEwbN81Oc9jxAOaX2SQHaE5\u0026pid=Api","contentUrl":"https://southerntrippers.com/wp-content/uploads/2022/08/things-to-do-in-atlanta-at-night-centennial-olympic-park-.jpg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.GDtNm5tAuiy9g1VcZR9skwHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/07/24-Things-to-Do-in-Downtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/things-to-do-in-downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.eWESOkT4ZSC4Ul01bDAxNgHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/20/64/67/2064672a8037327137c12d03d8b5cd75.jpg","hostPageUrl":"https://www.pinterest.com/pin/30-best-things-to-do-in-atlanta-from-art-museums-to-highend-shopping-in-2022--45739752457633108/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.8B9LmuRXqgsRTEY6n_RNlAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/06/The-Best-Things-to-Do-in-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-event-calendar/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.k7hGzw27Lx3uqX8C1eseEQHaE6\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2022/05/atlanta-1280x850.jpeg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VOzVfWJV7_6joNkGhLP7AAHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/ce/c5/25/cec525ba57be0022de5d70d1a1f3dd88.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/196891814951760239/18-cool-things-to-do-in-atlanta-georgia-on-your-first-visit/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6_fMiPMrfeqD9K0vgkdR6QHaE8\u0026pid=Api","contentUrl":"https://www.getawaymavens.com/wp-content/uploads/2019/01/Westin-Peachtree-Atlanta.jpg","hostPageUrl":"https://jetsetterjournals.com/unforgettable-adventures-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.sVk_ivij0dhmi2eo7Wy1IgHaFj\u0026pid=Api","contentUrl":"http://3.bp.blogspot.com/-1XOnrUou1lA/Utc7cr219LI/AAAAAAAAAZw/AsZhHck3E74/s1600/securedownload\u002B(11).jpg","hostPageUrl":"https://colourwery.weebly.com/blog/unique-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg?is-pending-load=1","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.9sNSjJkF0CqqlqsjR5kKzQHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/c2/8b/05/c28b05fc85f980ba25b12a2aa1ccd9d2.jpg","hostPageUrl":"https://www.pinterest.com/pin/53-things-to-do-in-atlanta-you-should-not-miss-from-locals-in-the-know--80924124545880293/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/Vl6GPUuj1mc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=Vl6GPUuj1mc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zDsChx3osL1eGz8a5zpM-AHaLH\u0026pid=Api","contentUrl":"https://localadventurer.com/wp-content/uploads/2016/04/free-things-to-do-in-atlanta.jpg","hostPageUrl":"https://localadventurer.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H24avZSeUDqe-Oc4AbvN-gHaLH\u0026pid=Api","contentUrl":"https://familydestinationsguide.com/wp-content/uploads/2020/11/Best-Things-to-Do-in-Atlanta-pinterest.jpg","hostPageUrl":"https://familydestinationsguide.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 25 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 25 2025"},"webPages":{"value":[{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. ... January 25 2025 Special event at the Atlanta Botanical Garden, with speakers and plant sale. More details \u2013 Gals on the Go, January 25 2025"},{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. Check out the complete schedule of this month\u2019s events. ... Prices from $25 Avg. price ~ $34. 42 tickets remaining! Tickets; Jan. 04. 2025. 7:00 PM Sat. Da\u0027Vinchi. ATL Comedy Theater. 30093, 4650 Jimmy Carter Boulevard, Norcross, GA, US","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"25 Reasons to Visit Atlanta in 2025","url":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025","snippet":"The Atlanta Chinese Dance Company performs on stage on Jan. 25,2025, at Aurora Children\u2019s Playhouse. Here are more ways to celebrate Chinese New Year in Atlanta. Spring Festivals. Atlanta Film Festival 2025 takes place April 24 through May 4, 2025, at the Plaza Theatre \u0026 Tara Theatre."},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Find cool things to do in Atlanta during the month of January, from annual festivals to seasonal activities, theatrical performances, and more! ... Atlanta Hawks vs. Toronto Raptors, Jan. 25 at 7:30 p.m. Atlanta Vibe Home Opener vs. Columbus Fury, Jan. 16 at 7 p.m. Atlanta Vibe vs. Grand Rapids Rise, Jan. 19 at 3 p.m. ... 2025 marks the 100th ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fun, Free Activities for Kids in January 2025: 15 Free Things To Do in ...","url":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15-free-things-to-do-in-atlanta-with-kids/ar-AA1mOEey","displayUrl":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15...","snippet":"Free Things to Do with Kids in Atlanta January 2025. Chick-fil-A Peach Bowl Fan Fest ... January 25, 2025. Atlanta Motor Speedway is inviting fans to come see the historic speedway for themselves ..."}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://www.experism.com/atlanta/january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.k7hGzw27Lx3uqX8C1eseEQHaE6\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2022/05/atlanta-1280x850.jpeg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.wsLrsO7mRA1lPtu-yRPZHQAAAA\u0026pid=Api","contentUrl":"https://secretatlanta.co/wp-content/uploads/2021/12/259158507_6597733293632503_4617562253295709670_n.jpg","hostPageUrl":"https://secretatlanta.co/things-to-do-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.3I8aZp7EItFo_du1fS5CVQHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/dc/81/14/dc8114905086f0025ffc20dfd179934b.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/649996158733928577/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.-1BVEwbN81Oc9jxAOaX2SQHaE5\u0026pid=Api","contentUrl":"https://southerntrippers.com/wp-content/uploads/2022/08/things-to-do-in-atlanta-at-night-centennial-olympic-park-.jpg","hostPageUrl":"https://nertqherminia.pages.dev/yfskiot-things-to-do-in-atlanta-georgia-in-january-2024-wwtcyrq/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.GDtNm5tAuiy9g1VcZR9skwHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/07/24-Things-to-Do-in-Downtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/things-to-do-in-downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://martellewgennie.pages.dev/ycqmcw-things-to-do-in-atlanta-this-weekend-events-dgpyjjsmr-photos/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.eWESOkT4ZSC4Ul01bDAxNgHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/20/64/67/2064672a8037327137c12d03d8b5cd75.jpg","hostPageUrl":"https://www.pinterest.com/pin/30-best-things-to-do-in-atlanta-from-art-museums-to-highend-shopping-in-2022--45739752457633108/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.9sNSjJkF0CqqlqsjR5kKzQHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/c2/8b/05/c28b05fc85f980ba25b12a2aa1ccd9d2.jpg","hostPageUrl":"https://www.pinterest.com/pin/53-things-to-do-in-atlanta-you-should-not-miss-from-locals-in-the-know--80924124545880293/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H24avZSeUDqe-Oc4AbvN-gHaLH\u0026pid=Api","contentUrl":"https://familydestinationsguide.com/wp-content/uploads/2020/11/Best-Things-to-Do-in-Atlanta-pinterest.jpg","hostPageUrl":"https://familydestinationsguide.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.8B9LmuRXqgsRTEY6n_RNlAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/06/The-Best-Things-to-Do-in-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-event-calendar/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQHaL5\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.artofit.org/image-gallery/34480753388071714/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/Vl6GPUuj1mc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=Vl6GPUuj1mc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.sVk_ivij0dhmi2eo7Wy1IgHaFj\u0026pid=Api","contentUrl":"http://3.bp.blogspot.com/-1XOnrUou1lA/Utc7cr219LI/AAAAAAAAAZw/AsZhHck3E74/s1600/securedownload\u002B(11).jpg","hostPageUrl":"https://colourwery.weebly.com/blog/unique-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zDsChx3osL1eGz8a5zpM-AHaLH\u0026pid=Api","contentUrl":"https://localadventurer.com/wp-content/uploads/2016/04/free-things-to-do-in-atlanta.jpg","hostPageUrl":"https://localadventurer.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fE1yxTuY24JK5qWh9l2dLQAAAA\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/8a/ab/6a/8aab6a9551de817b5f452e2dc01acf51.jpg","hostPageUrl":"https://www.pinterest.com/pin/312859505366250461/"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 4, 2025? https://atlanta-ga.events/january/"}
```
and got this result:
```
"Here are the events happening in Atlanta on January 4, 2025:\n\n1. **Titanic: An Immersive Voyage**\n   - **Time:** 3:30 AM\n   - **Location:** Hub Art Center Atlanta\n   - **Prices:** From $64, Avg. price ~ $77\n\n2. **Georgia Tech Yellow Jackets vs. Boston College Eagles**\n   - **Time:** 12:00 PM\n   - **Location:** McCamish Pavilion\n   - **Prices:** From $8, Avg. price ~ $49\n\n3. **SalgirahArts Festival: Expressions of Shukran**\n   - **Time:** 12:00 PM\n   - **Location:** Gas South Theater\n   - **Prices:** From $22, Avg. price ~ $43\n\n4. **Kennesaw State Owls vs. Jacksonville State Gamecocks**\n   - **Time:** 5:00 PM\n   - **Location:** KSU Convocation Center\n   - **Prices:** From $25, Avg. price ~ $33\n\n5. **Georgia State Panthers vs. Louisiana-Lafayette Ragin\u0027 Cajuns**\n   - **Time:** 7:00 PM\n   - **Location:** Georgia State Convocation Center\n   - **Prices:** From $10, Avg. price ~ $33\n\n6. **Canopy**\n   - **Time:** 7:00 PM\n   - **Location:** Altar at Masquerade\n   - **Prices:** From $23, Avg. price ~ $31\n\n7. **Kaya\u0027s Embrace**\n   - **Time:** 7:00 PM\n   - **Location:** Purgatory Stage at Masquerade\n   - **Prices:** From $25, Avg. price ~ $34\n\n8. **Da\u0027Vinchi**\n   - **Time:** 7:00 PM\n   - **Location:** ATL Comedy Theater\n   - **Prices:** From $59, Avg. price ~ $101\n\n9. **Georgia Swarm vs. Rochester Knighthawks**\n   - **Time:** 7:30 PM\n   - **Location:** Gas South Arena\n   - **Prices:** From $9, Avg. price ~ $48\n\n10. **SalgirahArts Festival: Expressions of Shukran**\n    - **Time:** 8:00 PM\n    - **Location:** Gas South Theater\n    - **Prices:** From $28, Avg. price ~ $32\n\n11. **Sonic Symphony Live**\n    - **Time:** 8:00 PM\n    - **Location:** Cobb Energy Performing Arts Centre\n    - **Prices:** From $125, Avg. price ~ $338\n\n12. **Best of Atlanta Comedy Showcase**\n    - **Time:** 8:30 PM\n    - **Location:** Helium Comedy Club - Atlanta\n    - **Prices:** From $24, Avg. price ~ $34\n\n13. **Club 90s: Justin Bieber Night**\n    - **Time:** 9:00 PM\n    - **Location:** Buckhead Theatre\n    - **Prices:** From $34, Avg. price ~ $40\n\nThese events cover a variety of activities from sports to concerts and festivals."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 11, 2025? https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html"}
```
and got this result:
```
"On January 11, 2025, several events are happening in Atlanta:\n\n1. **Joe Gatto: Let\u0027s Get Into It**\n   - **Type:** Comedy Show\n   - **Location:** Atlanta Symphony Hall\n   - **Details:** [More details](https://www.aso.org/events/detail/joe-gatto)\n\n2. **Last Podcast on the Left: JK Ultra Tour**\n   - **Type:** Comedy Show\n   - **Location:** Coca-Cola Roxy\n   - **Details:** [More details](https://concerts.livenation.com/last-podcast-on-the-left-jk-atlanta-georgia-01-11-2025/event/0E0061581C407C26)\n\n3. **Atlanta Travel \u0026 Adventure Show**\n   - **Type:** Travel and Tourism Trade Show\n   - **Location:** Georgia International Convention Center\n   - **Details:** [More details](https://travelshows.com/shows/atlanta/)\n\n4. **Love\u0027s Labour\u0027s Lost**\n   - **Type:** Play\n   - **Location:** Shakespeare Tavern Playhouse\n   - **Details:** [More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)\n\nThese events offer a variety of entertainment options, from comedy shows to theatrical performances."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 18, 2025? https://discoveratlanta.com/events/main/"}
```
and got this result:
```
"On January 18, 2025, the following events are happening in Atlanta:\n\n1. **AT\u0026T Playoff Playlist Live with Lil Wayne \u0026 More!**\n   - **Date:** January 18, 2025\n   - **Details:** A music event featuring performances by Lil Wayne and other artists.\n   - ![AT\u0026T Playoff Playlist Live with Lil Wayne \u0026 More!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734539959/EventEntry/vot9ijnqzxmxyqaz19s9.jpg)\n   - [More Info](https://discoveratlanta.com/event/detail/at-t-playoff-playlist-live-with-lil-wayne-more/)\n\n2. **CFP Taste of the Championship Atlanta**\n   - **Date:** January 19, 2025\n   - **Details:** A tasting event leading up to the College Football Playoff National Championship.\n   - ![CFP Taste of the Championship Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734464937/EventEntry/vk1qmw7hdetbaupmi5lu.jpg)\n   - [More Info](https://discoveratlanta.com/event/detail/cfp-taste-of-the-championship-atlanta/)\n\n3. **Georgia Tech Women\u0027s Basketball vs. Clemson**\n   - **Date:** January 19, 2025\n   - **Details:** A college basketball game.\n   - ![Georgia Tech Women\u0027s Basketball vs. Clemson](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696046/EventEntry/hkdj81zh4nsxsktxxegx.jpg)\n   - [More Info](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-clemson/)\n\nFor more detailed information on these events, you can visit [Discover Atlanta\u0027s events page](https://discoveratlanta.com/events/main/)."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 25, 2025? https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html"}
```
and got this result:
```
"On January 25, 2025, the following events are happening in Atlanta:\n\n1. **Run With the Dogs**: An annual 5K and pet races in Decatur to benefit the Decatur Bulldogs Boosters. [More details](http://www.rwtd5k.com)\n\n2. **Spring Gardening Symposium \u0026 Plant Sale**: A special event at the Atlanta Botanical Garden featuring speakers and a plant sale. [More details](https://atlantabg.org/calendar/spring-symposium-2025/)\n\n3. **Gals on the Go**: A live podcast show at Center Stage. [More details](https://www.centerstage-atlanta.com/events/gals-on-the-go/)\n\n4. **Maria Bamford**: Comedy show at the Buckhead Theatre. [More details](https://concerts.livenation.com/maria-bamford-atlanta-georgia-01-25-2025/event/0E00611B94352720)\n\n5. **Magician Rich Crawford**: An illusion show at Red Clay Music Foundry, Duluth. [More details](https://eddieowenpresents.freshtix.com/events/richcrawford)\n\n6. **Lunar New Year Festival**: An Asian-inspired festival at Stone Mountain Park, featuring a drone and light show, lighted parade, cultural craft activities, live performances, and other cultural activities. [More details](https://stonemountainpark.com/activity/events/lunar-new-year/)\n\nThese events showcase a variety of activities including fitness, gardening, entertainment, and cultural celebrations."
```

Here are some exciting things to do in Atlanta for each of the next four weekends in January 2025, along with pictures and links for more details.

### Weekend of January 4, 2025
1. **Titanic: An Immersive Voyage**
   - **Time:** 3:30 AM
   - **Location:** Hub Art Center Atlanta
   - **Prices:** From $64
   - ![Titanic: An Immersive Voyage](https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg)
   - [More Info](https://atlanta-ga.events/january/)

2. **Georgia Tech Yellow Jackets vs. Boston College Eagles**
   - **Time:** 12:00 PM
   - **Location:** McCamish Pavilion
   - **Prices:** From $8
   - ![Georgia Tech Yellow Jackets](https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg)

3. **Kennesaw State Owls vs. Jacksonville State Gamecocks**
   - **Time:** 5:00 PM
   - **Location:** KSU Convocation Center
   - **Prices:** From $25
   - ![Kennesaw State Owls](https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg)

4. **Georgia Swarm vs. Rochester Knighthawks**
   - **Time:** 7:30 PM
   - **Location:** Gas South Arena
   - **Prices:** From $9
   - ![Georgia Swarm](https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg)

### Weekend of January 11, 2025
1. **Joe Gatto: Let's Get Into It**
   - **Type:** Comedy Show
   - **Location:** Atlanta Symphony Hall
   - [More Info](https://www.aso.org/events/detail/joe-gatto)

2. **Last Podcast on the Left: JK Ultra Tour**
   - **Type:** Comedy Show
   - **Location:** Coca-Cola Roxy
   - [More Info](https://concerts.livenation.com/last-podcast-on-the-left-jk-atlanta-georgia-01-11-2025/event/0E0061581C407C26)

3. **Atlanta Travel & Adventure Show**
   - **Type:** Travel and Tourism Trade Show
   - **Location:** Georgia International Convention Center
   - [More Info](https://travelshows.com/shows/atlanta/)

4. **Love's Labour's Lost**
   - **Type:** Play
   - **Location:** Shakespeare Tavern Playhouse
   - [More Info](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)

### Weekend of January 18, 2025
1. **AT&T Playoff Playlist Live with Lil Wayne & More!**
   - **Date:** January 18, 2025
   - ![AT&T Playoff Playlist Live](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734539959/EventEntry/vot9ijnqzxmxyqaz19s9.jpg)
   - [More Info](https://discoveratlanta.com/event/detail/at-t-playoff-playlist-live-with-lil-wayne-more/)

2. **Georgia Tech Women's Basketball vs. Clemson**
   - **Date:** January 19, 2025
   - ![Georgia Tech Women's Basketball](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696046/EventEntry/hkdj81zh4nsxsktxxegx.jpg)
   - [More Info](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-clemson/)

### Weekend of January 25, 2025
1. **Run With the Dogs**
   - **Type:** Annual 5K and pet races
   - [More Info](http://www.rwtd5k.com)

2. **Spring Gardening Symposium & Plant Sale**
   - **Type:** Event at the Atlanta Botanical Garden
   - [More Info](https://atlantabg.org/calendar/spring-symposium-2025/)

3. **Gals on the Go**
   - **Type:** Live podcast show
   - [More Info](https://www.centerstage-atlanta.com/events/gals-on-the-go/)

4. **Lunar New Year Festival**
   - **Type:** Cultural festival at Stone Mountain Park
   - [More Info](https://stonemountainpark.com/activity/events/lunar-new-year/)

These events offer a variety of activities from sports to concerts and cultural celebrations, ensuring a fun-filled January in Atlanta!


## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebSearchAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebSearchAgentAnt");
}

assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebPageAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebPageAgentAnt");
}

Deleted assistant
Deleted assistant
